<a href="https://colab.research.google.com/github/necmigunduz/second_ai_try/blob/main/second_ai_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import torch
from torch import nn #PyTorch blocks for Neural Network
import matplotlib.pyplot as plt
torch.__version__

'2.3.1+cu121'

In [27]:
## Data: Preparing and loading
# Known parameters
weight = 0.7
bias = 0.3

start = 0
end = 1
step = 0.02

X = torch.arange(start, end, step).unsqueeze(dim=1)

# Create a line function
y = bias + weight * X

# Length
len(X), len(y)

(50, 50)

In [28]:
### Split data into training and test sets
